In [1]:
import pathlib
from xclim import subset
import xarray
import geopandas as gpd
from shapely.geometry import Point

In [8]:
import numpy as np

In [22]:
import pandas as pd

In [2]:
data = pathlib.Path("/home/mlopez/EXEC/Processed Data Varibales/NorESM1-M_rcp85_tg_mean_annual.nc")
json = pathlib.Path("/home/mlopez/EXEC/Shapefiles/territoire_guide.shp")

In [3]:
shape = gpd.read_file(json)
shape

,TER_GUIDE,geometry
0,1a,"MULTIPOLYGON (((-324035.000 263596.000, -32301..."
1,2a,"POLYGON ((-581072.614 306295.338, -581104.138 ..."
2,2b,"MULTIPOLYGON (((-175814.000 338636.000, -17530..."
3,2c,"POLYGON ((-302608.998 192261.534, -302594.079 ..."
4,3ab,"POLYGON ((-794232.694 425801.915, -794182.539 ..."
5,3c,"POLYGON ((-273806.643 347134.759, -273734.686 ..."
6,3d,"POLYGON ((-85261.812 342007.834, -85368.000 34..."
7,4a,"POLYGON ((-814550.882 517597.409, -813875.987 ..."
8,4bc,"POLYGON ((-613258.856 484779.491, -613502.910 ..."
9,4de,"POLYGON ((-294234.862 579991.582, -294413.155 ..."


In [9]:
ds = xarray.open_dataset(data)
ds

<xarray.Dataset>
Dimensions:  (lat: 320, lon: 416, time: 151)
Coordinates:
  * time     (time) object 1950-01-01 00:00:00 ... 2100-01-01 00:00:00
  * lon      (lon) float32 -89.04521 -88.96188 -88.87855 ... -54.54659 -54.46326
  * lat      (lat) float32 66.62331 66.53998 66.45665 ... 40.12437 40.04104
Data variables:
    tg_mean  (time, lat, lon) float32 ...
Attributes:
    Conventions:     CF-1.5
    title:           NorESM1-M model output prepared for CMIP5 historical
    history:         2011-06-01T01:20:15Z CMOR rewrote data to comply with CF...
    institution:     Norwegian Climate Centre
    source:          NorESM1-M 2011  atmosphere: CAM-Oslo (CAM4-Oslo-noresm-v...
    redistribution:  Redistribution prohibited. For internal use only.

In [26]:
nds = ds.indexes['time'].to_datetimeindex()
nds

DatetimeIndex(['1950-01-01', '1951-01-01', '1952-01-01', '1953-01-01',
               '1954-01-01', '1955-01-01', '1956-01-01', '1957-01-01',
               '1958-01-01', '1959-01-01',
               ...
               '2091-01-01', '2092-01-01', '2093-01-01', '2094-01-01',
               '2095-01-01', '2096-01-01', '2097-01-01', '2098-01-01',
               '2099-01-01', '2100-01-01'],
              dtype='datetime64[ns]', length=151, freq=None)

In [28]:
ds['time'] = nds
ds

<xarray.Dataset>
Dimensions:  (lat: 320, lon: 416, time: 151)
Coordinates:
  * time     (time) datetime64[ns] 1950-01-01 1951-01-01 ... 2100-01-01
  * lon      (lon) float32 -89.04521 -88.96188 -88.87855 ... -54.54659 -54.46326
  * lat      (lat) float32 66.62331 66.53998 66.45665 ... 40.12437 40.04104
Data variables:
    tg_mean  (time, lat, lon) float32 260.48712 260.5821 260.5915 ... nan nan
Attributes:
    Conventions:     CF-1.5
    title:           NorESM1-M model output prepared for CMIP5 historical
    history:         2011-06-01T01:20:15Z CMOR rewrote data to comply with CF...
    institution:     Norwegian Climate Centre
    source:          NorESM1-M 2011  atmosphere: CAM-Oslo (CAM4-Oslo-noresm-v...
    redistribution:  Redistribution prohibited. For internal use only.

In [29]:
df = ds.to_dataframe().reset_index()
df

,lat,lon,time,tg_mean
0,66.623306,-89.045212,1950-01-01,260.487122
1,66.623306,-89.045212,1951-01-01,259.087128
2,66.623306,-89.045212,1952-01-01,258.688416
3,66.623306,-89.045212,1953-01-01,257.983765
4,66.623306,-89.045212,1954-01-01,259.069092
...,...,...,...,...
20101115,40.041039,-54.463261,2096-01-01,NaN
20101116,40.041039,-54.463261,2097-01-01,NaN
20101117,40.041039,-54.463261,2098-01-01,NaN
20101118,40.041039,-54.463261,2099-01-01,NaN


In [12]:
geom = [Point(x, y) for x, y in zip(df['lon'], df['lat'])]
gdf = gpd.GeoDataFrame(df, geometry=geom)
gdf

,lat,lon,time,tg_mean,geometry
0,66.623306,-89.045212,1950-01-01 00:00:00,260.487122,POINT (-89.04521 66.62331)
1,66.623306,-89.045212,1951-01-01 00:00:00,259.087128,POINT (-89.04521 66.62331)
2,66.623306,-89.045212,1952-01-01 00:00:00,258.688416,POINT (-89.04521 66.62331)
3,66.623306,-89.045212,1953-01-01 00:00:00,257.983765,POINT (-89.04521 66.62331)
4,66.623306,-89.045212,1954-01-01 00:00:00,259.069092,POINT (-89.04521 66.62331)
...,...,...,...,...,...
20101115,40.041039,-54.463261,2096-01-01 00:00:00,NaN,POINT (-54.46326 40.04104)
20101116,40.041039,-54.463261,2097-01-01 00:00:00,NaN,POINT (-54.46326 40.04104)
20101117,40.041039,-54.463261,2098-01-01 00:00:00,NaN,POINT (-54.46326 40.04104)
20101118,40.041039,-54.463261,2099-01-01 00:00:00,NaN,POINT (-54.46326 40.04104)


In [ ]:
gdf.to_file("tgmenaannuel.shp")

In [ ]:
data2 = data = pathlib.Path("tgmenaannuel.shp")

In [19]:
ss = subset.subset_shape(json, data2, start_date='1950-01-01', end_date='1952-01-01')
ss

DriverError: unsupported driver: 'netCDF'

In [ ]:

for i in ss:
    ds = xarray.open_dataset(i)
    df = ds.to_dataframe().reset_index()
    geom = [Point(x, y) for x, y in zip(df['lon'], df['lat'])]
    gdf = gpd.GeoDataFrame(df, geometry=geom)
    gdf.to_json(f"{i}.json")